# Parse NOC to ISO data from wikipedia
Because NOC in dataframe differentiate t.ex Soviet Union, EUN, and Russia, we prefer to use ISO to plot on map. For Soviet Union, EUN and Russia, they have the same country: Russia. 

We therefore parse noc to iso dataframe from wikipedia.
However, one question is to get red of the reference number in the country column.

We therefore apply the following method:
[Reference][link]: Pandas read_html() ignore superscripts and subscripts

[link]:https://stackoverflow.com/questions/64309887/pandas-read-html-ignore-superscripts-and-subscripts

In [15]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://simple.wikipedia.org/wiki/Comparison_of_IOC,_FIFA,_and_ISO_3166_country_codes'

soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# remove <sup>
for sup in soup.select('sup'):
    sup.extract()

table = pd.read_html(str(soup))[0]
table.head()

,Flag,Country,IOC,FIFA,ISO
0,NaN,Afghanistan,AFG,AFG,AFG
1,NaN,Åland Islands,NaN,NaN,ALA
2,NaN,Albania,ALB,ALB,ALB
3,NaN,Algeria,ALG,ALG,DZA
4,NaN,American Samoa,ASA,ASA,ASM


In [16]:
# select relevant columns
table = table.iloc[:, [1, 2, 4]]
table = table.rename(columns={"IOC": "NOC"})
noc_iso = table[table["NOC"].notna()]
noc_iso.head()

,Country,NOC,ISO
0,Afghanistan,AFG,AFG
2,Albania,ALB,ALB
3,Algeria,ALG,DZA
4,American Samoa,ASA,ASM
5,Andorra,AND,AND


- add rows to dataframe
- where both URS(soviet union), EUN([Unified Team at the Olympics][link]) are the regions Russia

[link]: https://en.wikipedia.org/wiki/Unified_Team_at_the_Olympics

In [17]:
data = {'Country': ['Russia', 'Russia'],
	'NOC': ['URS', 'EUN'],
	'ISO': ['RUS', 'RUS']}
new_row = pd.DataFrame(data)
noc_iso = noc_iso.append(new_row, ignore_index=True)
noc_iso.tail()

,Country,NOC,ISO
202,Yemen,YEM,YEM
203,Zambia,ZAM,ZMB
204,Zimbabwe,ZIM,ZWE
205,Russia,URS,RUS
206,Russia,EUN,RUS


In [18]:
noc_iso.to_csv("data/noc_iso.csv")

In [22]:
# Import data
athlete_regions = pd.read_csv("data/athlete_regions.csv")
athlete_regions = athlete_regions.iloc[:, :-2]
athlete_regions.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands


In [23]:
# get noc_iso data
noc_iso = pd.read_csv("data/noc_iso.csv")
noc_iso = noc_iso.iloc[:, 1:]
noc_iso.tail()

,Country,NOC,ISO
202,Yemen,YEM,YEM
203,Zambia,ZAM,ZMB
204,Zimbabwe,ZIM,ZWE
205,Russia,URS,RUS
206,Russia,EUN,RUS


In [24]:
athlete_iso = athlete_regions.merge(noc_iso, on="NOC", how="left")
athlete_iso = athlete_iso.sort_values(by=["NOC"])
athlete_iso[athlete_iso["NOC"]=="URS"]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,Country,ISO
31740,16301,Andrey Anatolyevich Bukin,M,30.0,185.0,63.0,Soviet Union-1,URS,1988 Winter,1988,Winter,Calgary,Figure Skating,Figure Skating Mixed Ice Dancing,Gold,Russia,Russia,RUS
13023,7084,Vera Vasilyevna Baklanova,F,21.0,160.0,57.0,Soviet Union,URS,1968 Summer,1968,Summer,Mexico City,Diving,Diving Women's Springboard,NaN,Russia,Russia,RUS
267011,133589,Aleksandr Nikolayevich Zabelin,M,28.0,168.0,72.0,Soviet Union,URS,1960 Summer,1960,Summer,Roma,Shooting,"Shooting Men's Rapid-Fire Pistol, 25 metres",Bronze,Russia,Russia,RUS
247637,123987,Igor Anatolyevich Valetov,M,26.0,NaN,NaN,Soviet Union,URS,1972 Summer,1972,Summer,Munich,Fencing,"Fencing Men's epee, Individual",NaN,Russia,Russia,RUS
13022,7084,Vera Vasilyevna Baklanova,F,17.0,160.0,57.0,Soviet Union,URS,1964 Summer,1964,Summer,Tokyo,Diving,Diving Women's Springboard,NaN,Russia,Russia,RUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145804,73171,Baldadash Madatov,M,NaN,NaN,NaN,Soviet Union,URS,1952 Summer,1952,Summer,Helsinki,Athletics,Athletics Men's Long Jump,NaN,Russia,Russia,RUS
144857,72696,Aleksandr Lyubaturov,M,30.0,NaN,NaN,Soviet Union,URS,1972 Summer,1972,Summer,Munich,Rowing,Rowing Men's Coxed Fours,NaN,Russia,Russia,RUS
144862,72699,Ivan Lyubimov,M,27.0,167.0,NaN,Soviet Union,URS,1960 Winter,1960,Winter,Squaw Valley,Cross Country Skiing,Cross Country Skiing Men's 50 kilometres,NaN,Russia,Russia,RUS
147131,73845,Andrey Gennadyevich Makeyev,M,24.0,197.0,92.0,Soviet Union,URS,1976 Summer,1976,Summer,Montreal,Basketball,Basketball Men's Basketball,Bronze,Russia,Russia,RUS


In [27]:
# save athletes_iso
athlete_iso.to_csv("data/athlete_iso.csv")